In [ ]:
import numpy as np
import pandas as pd
import math
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
col = ['sepal_length','sepal_width','petal_length','petal_width']
df = pd.DataFrame(load_iris().data, columns=col)
df['type'] = 0
df.iloc[0:50,-1] = 0
df.iloc[50:100,-1] = 1
df.iloc[100:150,-1] = 2

In [ ]:
class Node():
  def __init__(self, fi=None, ts=None, left=None, right=None, ig=None, value=None):
    '''Constructor of class, __init__ gives values to node as per attributes it have
    here, value is for leafs and other are for conditional node
    '''
    self.right = right
    self.left = left
    self.fi = fi
    self.ts = ts
    self.ig = ig
    self.value = value

In [ ]:
class DTC():
  def __init__(self, min_splt=2, max_depth=2):
    '''Here, constructing decision tree where minimum split default value and 
    max depth default values are given for conditional block initial conditions
    '''
    self.root = None
    self.min_splt = min_splt
    self.max_depth = max_depth
  def treeBuilderMain(self, dataset, depthNow=0):
    X = dataset[:,:-1]
    y = dataset[:,-1]
    '''Here, Features are in X variable which is whole dataframe except last column
    which is type(species) column.
    y is type column which we need to predict using decision tree
    X and y are defined to make decisions based on label and features
    '''
    sam, fet = np.shape(X)
    '''shape of features is given to sam and fet variables to limit conditionals
    as per dataset length
    '''
    if sam>=self.min_splt and depthNow<=self.max_depth:
      '''using opt_split_select function to get data of most optimized split for
      tree creation (Final Tree)
      '''
      opt_splt = self.opt_split_select(dataset, sam, fet)
      '''When info gain reach 0, we consider that Node as pure Node and leave 
      it without any further process
      Hence whenever we find info gain more than 0, we keep processing on node 
      for more splits (Done by splt function)
      '''
      if opt_splt['ig']>0:
        left = self.treeBuilderMain(opt_splt['datasetLeft'], depthNow+1)
        right = self.treeBuilderMain(opt_splt['datasetRight'], depthNow+1)
        return Node(opt_splt['fi'], opt_splt['ts'],left, right, opt_splt['ig'])
    leafVal = self.leafValue(y)
    return Node(value=leafVal)

  def opt_split_select(self, dataset, sam, fet):
    '''Creating split for optimized tree
    '''
    opt_splt = {}
    max_ig = -float("inf")

    for fi in range(fet):
      f_val = dataset[:,fi]
      t = np.unique(f_val)
      for ts in t:
        datasetLeft, datasetRight = self.splt(dataset, fi, ts)
        if len(datasetLeft)>0 and len(datasetRight)>0:
          y, yLeft, yRight = dataset[:, -1], datasetLeft[:, -1], datasetRight[:, -1]
          iNow = self.ig(y, yLeft, yRight, "gini")
          if iNow>max_ig:
            opt_splt['fi']=fi
            opt_splt['ts']=ts
            opt_splt['datasetLeft']=datasetLeft
            opt_splt['datasetRight']=datasetRight
            opt_splt['ig']=iNow
            max_ig = iNow
    return opt_splt

  def splt(self, dataset, fi, ts):
    datasetLeft = np.array([row for row in dataset if row[fi]<=ts])
    datasetRight = np.array([row for row in dataset if row[fi]>ts])
    return datasetLeft, datasetRight

  def ig(self, p, l, r, mode="entropy"):
    wl = len(l)/len(p)
    wr = len(r)/len(p)
    if mode=="gini":
      et = self.gini(p) - (wl*self.gini(l) + wr*self.gini(r))
    else:
      et = self.entropy(p) - (wl*self.entropy(l) + wr*self.entropy(r))
    return et

  def gini(self, y):
    labels = np.unique(y)
    gini_var = 0
    for cls in labels:
      prt = len(y[y == cls]) / len(y)
      gini_var += prt**2
    return 1 - gini_var
  
  def entropy(self, y):
    label = np.unique(y)
    entropy_var = 0
    for cls in class_label:
      prt = len(y[y == cls]) / len(y)
      entropy_var += -prt * np.log2(prt)
    return entropy_var

  def leafValue(self, Y):
    #compute leaf node
    Y = list(Y)
    return max(Y, key=Y.count)

  def fit(self, X, y):
    #now we train the tree
    dataset = np.concatenate((X, y), axis=1)
    self.root = self.treeBuilderMain(dataset)
  
  def predict(self, X):
    #now we predict on test tree
    pre = [self.prediction(x, self.root) for x in X]
    return pre
  def prediction(self, x, tree):
    if tree.value!=None: 
      return tree.value
    f_val = x[tree.fi]
    if f_val<=tree.ts:
      return self.prediction(x, tree.left)
    else:
      return self.prediction(x, tree.right)

In [ ]:
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values.reshape(-1,1)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.2, random_state=100)

In [ ]:
classifier = DTC(min_splt=3, max_depth=3)
classifier.fit(X_train,Y_train)

In [ ]:
Y_pred = classifier.predict(X_test)
print('Accuracy=', accuracy_score(Y_test, Y_pred)) 


Accuracy= 0.9666666666666667
